# Vaje 4 - optimizacija hiperparametrov s knjižnico hyperopt

A.1 Hyperopt je nekoliko bolj zapletena optimizacija, zato bomo za začetek poskusili najti x, ki minimizira funkcijo
   optimalne f(x) = x^2. Kandidate za ta x bomo iskali na intervalu
   a) [-4, 4]
   b) [4, 8].
Oglejte si se druge moznosti zrebanja po prostoru: https://github.com/hyperopt/hyperopt/wiki/FMin (razdelek 2.1)

In [52]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from hyperopt import hp, tpe, rand, fmin, Trials, space_eval
from hyperopt import pyll, base
import numpy as np

prostorA = {"x": hp.uniform("x", -4, 4)}
prostorB = {"x": hp.uniform("x", 4, 8)}


def kriterijska_funkcija(parametri):
    x = parametri["x"]
    return x ** 2


prostor = prostorB
N = 50

trials = Trials()
best = fmin(fn=kriterijska_funkcija,
            space=prostor,
            algo=tpe.suggest,  # rand.suggest,
            max_evals=N,
            trials=trials)

best = space_eval(prostor, best)
best_value = kriterijska_funkcija(best)
# vse vrednosti paramtrov in kriterijske funkcije, ki smo jih preizkusili
xs = [trial["misc"]["vals"]["x"][0] for trial in trials.trials]
ys = [trial["result"]["loss"] for trial in trials.trials]
print(best, best_value)

100%|██████████| 50/50 [00:00<00:00, 140.83trial/s, best loss: 16.027318607995404]
{'x': 4.003413369612911} 16.027318607995404


A.2 Pogosto je treba najti več kot en optimalni parameter. Minimiziraj funkcijo $f(x, y) = (x - y)^2 + (y - 1)^2$.
   Prostora ne preiskuj enakomerno, ampak z normalno porazdelitvijo.

In [56]:
prostor_xy = {"x": hp.normal("x", 0, 3), "y": hp.normal("y", 0, 3)}

def kriterijska_funkcija_1(parametri):
    x = parametri["x"]
    y = parametri["y"]
    return (x - y) ** 2 + (y - 1) ** 2.

N = 100
trials = Trials()
best = fmin(fn=kriterijska_funkcija_1,
            space=prostor_xy,
            algo=tpe.suggest,  # rand.suggest
            max_evals=N,
            trials=trials)

best = space_eval(prostor_xy, best)
best_value = kriterijska_funkcija_1(best)
print(best, best_value)

100%|██████████| 100/100 [00:00<00:00, 163.30trial/s, best loss: 0.0013624144551192083]
{'x': 1.041887218845584, 'y': 1.0053688718036493} 0.0013624144551192083


A.3. Včasih moramo izvesti tudi diskretno izbiro, npr. ko izbiramo algoritem. Za spodnje podatke najdi najprimernejsi
   algoritem. Izbiraj med kNN, odlocitvenim drevesom in metodo podpornih vektorjev.

In [86]:
n_primerov = 1000
n_znacilk = 5

x = np.random.rand(n_primerov, n_znacilk)
y = np.dot(x, list(range(n_znacilk)))

povprecje = np.mean(y)
pozitivni = y >= povprecje
y[pozitivni] = 1
y[~pozitivni] = 0

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

prostor_algo = {"algo": hp.choice("algo", ["knn", "drevo", "svm"])}

np.random.seed(0)
x_ucna, x_testna, y_ucna, y_testna = train_test_split(x, y, test_size=0.2)


def kriterijska_funkcija_algo(parametri):
    a = parametri["algo"]
    if a == "knn":
        model = KNeighborsClassifier()
    elif a == "drevo":
        model = DecisionTreeClassifier()
    elif a == "svm":
        model = SVC()
    else:
        raise ValueError(f"Napacen algoritem: {a}")
    model.fit(x_ucna, y_ucna)
    y_napoved = model.predict(x_testna)
    return 1 - accuracy_score(y_testna, y_napoved)

N = 10
trials = Trials()
best = fmin(fn=kriterijska_funkcija_algo,
            space=prostor_algo,
            algo=tpe.suggest,  # rand.suggest
            max_evals=N,
            trials=trials)

best = space_eval(prostor_algo, best)
best_value = kriterijska_funkcija_algo(best)

xs = [trial["misc"]["vals"] for trial in trials]
acc = [trial["result"]["loss"] for trial in trials]
print(xs)
print(acc)
print(best, best_value)


100%|██████████| 10/10 [00:00<00:00, 37.69trial/s, best loss: 0.050000000000000044]
[{'algo': [0]}, {'algo': [1]}, {'algo': [2]}, {'algo': [2]}, {'algo': [1]}, {'algo': [0]}, {'algo': [2]}, {'algo': [1]}, {'algo': [1]}, {'algo': [2]}]
[0.09999999999999998, 0.135, 0.050000000000000044, 0.050000000000000044, 0.12, 0.09999999999999998, 0.050000000000000044, 0.14, 0.135, 0.050000000000000044]
{'algo': 'svm'} 0.050000000000000044


3. Poleg samega algoritma želimo iskati tudi optimalne vrednosti hiperparametrov. Seveda so hiperparametri odvisni od
   samega algoritma, zato je preiskovani prostor treba definirati gnezdeno, kot kaze primer.
   Razsiri preiskovani prostor tako, da dodas se kaksen parameter. Priporocamo, da jih poimenujes tako, kot so
   poimenovani pripadajoci argumenti v konstruktorju za dani model (npr. eden od parametrov za drevo je max_depth,
   ki pove, katera je najvecja dovoljena globina drevesa)

   Razsiri gnezdeni_prostor_spodaj.

In [3]:
gnezdeni_prostor = {
    "algo": hp.choice('algo', [
        {
            'ime': 'drevo',
            'max_depth': hp.choice('max_depth', [2, 4, 8, 16, 32]),
            'criterion': hp.choice('criterion', ["gini", "entropy"]),
            'min_samples_split': hp.qloguniform('min_samples_split', 1, 8, 1)
            # dodaj se kak parameter :)
        },
        {
            'ime': 'knn',
            'n_neighbors': hp.choice("n_neighbors", [1, 2, 3, 4, 5, 10, 15])
        },
        {
            'ime': 'svm',
            'C': hp.lognormal('C', 0, 1),
            'kernel': hp.choice('kernel', [
                {
                    'tip': 'linear'  # linearno
                },
                {
                    'tip': 'rbf',  # radialno
                    'gamma': hp.lognormal('gamma', 0, 1)
                },
                {
                    'tip': 'poly',  # polinomsko
                    'degree': hp.choice("degree", [1, 2, 3])
                }
            ]),
        },
    ])
}

4. Kako dobre so prednastavljene vrednosti? Primerjaj rezultate, ki jih dobis s preiskovanjem prostorov
   - prostor_algo (tu bodo pri algoritmih vedno uporabljene prednastavljene vrednosti)
   - gnezdeni_prostor

   Kateri algoritem (in katera konfiguracija parametrov) je najboljsa za dane podatke?
   Preizkusi na vseh treh moznih podatkovjih.

In [93]:

def izracunaj_y(xs, tip_podatkov):
    if tip_podatkov not in [1, 2, 3]:
        raise ValueError("Tip podatkov mora biti element [1, 2, 3].")
    if tip_podatkov == 1:
        r = list(range(xs.shape[1]))
        cs = np.array(r).reshape((-1, 1))
        ys = np.dot(xs, cs).reshape((-1,))
        poz = ys >= np.mean(ys)
        ys = np.array(["a" if p else "b" for p in poz])
    elif tip_podatkov == 2:
        group1 = xs[:, 1] > 0.5
        group2 = (xs[:, 1] <= 0.5) & (xs[:, 2] > 0.2)
        group3 = (xs[:, 1] <= 0.5) & (xs[:, 2] <= 0.2)
        ys = np.zeros(xs.shape[0], dtype=str)
        ys[group1] = "a"
        ys[group2] = "b"
        ys[group3] = "c"
    else:
        circle = np.sum(np.square(xs), axis=1) > 0.4
        ys = np.zeros(xs.shape[0], dtype=str)
        ys[circle] = "a"
        ys[~circle] = "b"
    return ys


def pripravi_podatke(n_ucna, n_test, n_znacilke, p_sum, tip_podatkov):
    x = np.random.rand(n_ucna + n_test, n_znacilke)
    y = izracunaj_y(x, tip_podatkov)
    vrednosti = list(set(y))
    for i in range(len(y)):
        # zamenjamo vrednost
        if np.random.rand(1) < p_sum:
            i0 = vrednosti.index(y[i])
            y[i] = vrednosti[(i0 + 1) % len(vrednosti)]
    x_ucna = x[:n_ucna]
    y_ucna = y[:n_ucna]
    x_test = x[n_ucna:]
    y_test = y[n_ucna:]
    return x_ucna, x_test, y_ucna, y_test


x_ucna, x_testna, y_ucna, y_testna = pripravi_podatke(1000, 200, 5, 0.1, 2)

def kriterijska_funkcija_privzete(parametri):
    a = parametri["algo"]
    if a == "knn":
        model = KNeighborsClassifier()
    elif a == "drevo":
        model = DecisionTreeClassifier()
    elif a == "svm":
        model = SVC()
    else:
        raise ValueError(f"Napacen algoritem: {a}")
    model.fit(x_ucna, y_ucna)
    y_napoved = model.predict(x_testna)
    return 1 - accuracy_score(y_testna, y_napoved)

def kriterijska_funkcija_razsirjeno(parametri):
    a = parametri["algo"]
    ime_algoritma = a["ime"]
    if ime_algoritma == "knn":
        model = KNeighborsClassifier(n_neighbors=a["n_neighbors"])
    elif ime_algoritma == "drevo":
        model = DecisionTreeClassifier(criterion=a["criterion"], 
                                       max_depth=a["max_depth"], 
                                       min_samples_split=int(a["min_samples_split"]))
    elif ime_algoritma == "svm":
        C = a["C"]
        kernel = a["kernel"]["tip"]
        # gamma in degree moramo definirati v vseh treh primerih: tam, kjer nista vazni, ju damo na 1
        neumna_vrednost = 1
        if kernel == "rbf":
            gamma = a["kernel"]["gamma"]
            degree = neumna_vrednost
        elif kernel == "linear":
            degree = neumna_vrednost
            gamma = neumna_vrednost
        else:
            gamma = neumna_vrednost
            degree = a["kernel"]["degree"]
        model = SVC(kernel=kernel, gamma=gamma, C=C, degree=degree)
    else:
        raise ValueError("Napacne nastavitve!")
    model.fit(x_ucna, y_ucna)
    y_hat = model.predict(x_testna)
    return 1 - accuracy_score(y_testna, y_hat)



trials_privzete = Trials()
best_privzete = fmin(fn=kriterijska_funkcija_privzete,
            space=prostor_algo,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials_privzete)
best_privzete = space_eval(prostor_algo, best_privzete)
best_value_privzete = kriterijska_funkcija_privzete(best_privzete)

trials = Trials()
best = fmin(fn=kriterijska_funkcija_razsirjeno,
            space=gnezdeni_prostor,
            algo=tpe.suggest,
            max_evals=500,
            trials=trials)
best = space_eval(gnezdeni_prostor, best)
best_value = kriterijska_funkcija_razsirjeno(best)


print("Privzete vrednosti")
print(best_privzete, best_value_privzete)
print("\nOptimizirane vrednosti")
print(best, best_value)



100%|██████████| 500/500 [00:24<00:00, 20.79trial/s, best loss: 0.08999999999999997]
Privzete vrednosti
{'algo': 'svm'} 0.13

Optimizirane vrednosti
{'algo': {'criterion': 'entropy', 'ime': 'drevo', 'max_depth': 32, 'min_samples_split': 512.0}} 0.08999999999999997
